In [ ]:
# Tuodaan PuLP kirjasto
from pulp import *
# Jos tulee virheilmoitus niin anna ensin komento "pip install pulp"

#Tuotantokapasiteetit tehtailla
kapasiteetit=[10200, 2000, 4100]

#Kysynnät asiakkailla
kysynnat = [650,260,650,130,780,2500,910,3120,910,3640,1040,650,780]

#Indeksit
tehtaat = range(3)
varastot= range(4)
asiakkaat = range(len(kysynnat))

kuljetuskustannuksetA=[[53,95,136,160],
[60,120,132,140],
[210, 190, 89, 71]]

kuljetuskustannuksetB = [[100, 120, 150, 160, 300, 310, 340, 490, 430,360, 280, 350, 200],
[200, 240, 280, 310, 280, 400, 440, 410, 380, 190, 80, 150, 90],
[280, 260, 320, 400, 140, 319, 290, 240, 230, 80, 60, 60, 160],
[320, 280, 200, 130, 130, 70, 100, 90, 	100, 190, 370, 320, 390]]


#Luodaan (minimoiva) lineaarinen optimointi malli PuLP:ssa
malli = LpProblem("Kuljetusmalli", LpMinimize)

#Luodaan päätösmuuttujat x_ij: "kuljetetaan x_ij tonnia kartonkia tehtaalta i varastoon j"
x = LpVariable.dicts("ReittiA",(tehtaat,varastot),0,None)
#Luodaan päätösmuuttujat y_jk: "kuljetetaan y_jk tonnia kartonkia varastosta j asiakkaalle k"
y = LpVariable.dicts("ReittiB",(varastot,asiakkaat),0,None)

#Lisätään kohdefunktio eli summa kaikkien reittien kuljetuskustannuksista
malli += (lpSum([kuljetuskustannuksetA[i][j]*x[i][j] for i in tehtaat for j in varastot]) + lpSum([kuljetuskustannuksetB[j][k]*y[j][k] for j in varastot for k in asiakkaat]), "Kuljetuskustannukset")

#Tuotantokapasiteettirajoitteet tehtailla
for i in tehtaat:
    malli += (lpSum([x[i][j] for j in varastot])<=kapasiteetit[i], "Tuotanto_tehtaalla_%s"%(i+1))

#Tasapainorajoitteet varastoilla
#
#

#Kysyntärajoitteet asiakkailla
#
#

#Ratkaistaan malli, eli etsitään optimaalinen ratkaisu (tai todetaan että mallille ei ole käypää ratkaisua)     
malli.solve()

# Tulostetaan päätösmuuttujien arvot optimiratkaisussa
kohdefunktionarvo = 0
for i in tehtaat:
    for j in varastot:
        if (x[i][j].varValue>0):
            print("Kuljetetaan %s tonnia kartonkia tehtaalta %d varastolle %a." % (x[i][j].varValue, (i+1), (j+1))) 
        kohdefunktionarvo = kohdefunktionarvo + kuljetuskustannuksetA[i][j]*x[i][j].varValue
    print("---")

print("-----------")
for j in varastot:
    for k in asiakkaat:
        if (y[j][k].varValue>0):
            print("Kuljetetaan %s tonnia kartonkia varastolta %d asiakkaalle %a." % (y[j][k].varValue, (j+1), (k+1))) 
        kohdefunktionarvo = kohdefunktionarvo + kuljetuskustannuksetB[j][k]*y[j][k].varValue
    print("---")
    
# Tulostetaan optimaalinen kohdefunktion arvo 
print("Kokonaiskuljetuskustannut = %s " % kohdefunktionarvo)

